####Crear tabla results_movie en capa Gold

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
spark.sql("""
          create table if not exists movie_gold.results_movie
          (
              year_release_date int,
              country_name string,
              company_name string,
              budget float,
              revenue float,
              movie_id int,
              country_id int,
              company_id int,
              created_date date,
              updated_date date
          )
          using delta
          """)

In [0]:
spark.sql (f"""
    create or replace temp view v_results_movie
    as
    select distinct m.year_release_date, c.country_name, pco.company_name, m.budget, m.revenue, m.movie_id, c.country_id, pco.company_id
    from movie_silver.movies m
    inner join movie_silver.productions_countries pc on m.movie_id = pc.movie_id
    inner join movie_silver.countries c on c.country_id = pc.country_id
    inner join movie_silver.movies_companies mc on mc.movie_id = m.movie_id
    inner join movie_silver.productions_companies pco on pco.company_id = mc.company_id
    where m.file_date = '{v_file_date}'
""")

In [0]:
%sql
select * from v_results_movie;

In [0]:
spark.sql(f"""
--hacemos merge de las vistas para crear una nueva tabla para indicar cuando se haga match, se hace una actualización de los registros y si no existe conincidencia se inserta un nuevo registro
    MERGE INTO movie_gold.results_movie tgt
    USING v_results_movie src --es una vista del día1
    ON (tgt.movie_id = src.movie_id and tgt.country_id = src.country_id and tgt.company_id = src.company_id) --columnas de coincidencia entre las tablas origen y destino
    WHEN MATCHED THEN
    UPDATE SET
        tgt.year_release_date = src.year_release_date,
        tgt.country_name = src.country_name,
        tgt.company_name = src.company_name,
        tgt.budget = src.budget,
        tgt.revenue = src.revenue,
        tgt.updated_date = current_timestamp
    WHEN NOT MATCHED
    THEN INSERT (
        year_release_date,
        country_name,
        company_name,
        budget,
        revenue,
        movie_id,
        country_id,
        company_id,
        created_date
    )
    VALUES (
        year_release_date,
        country_name,
        company_name,
        budget,
        revenue,
        movie_id,
        country_id,
        company_id,
        current_timestamp
    )
  """)

In [0]:
%sql
select count(1) from v_results_movie;

In [0]:
%sql
select count(1) from movie_gold.results_movie;